In [1]:
!pip install katna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 778.8 kB/s eta 0:00:00


In [9]:
import pandas as pd
import json

def clean_data_labels(data_path, label_json_path, output_path = None):
  def cleaning(label):
    label = label.replace('"', '').replace(' ', '')
    return ';'.join([id_label_map[i] for i in label.split(';')])

  df = pd.read_csv(data_path)

  with open(label_json_path, 'r') as file:
      data = json.load(file)

  id_label_map = {dictionary['id'] : dictionary['name'] for dictionary in data}

  df['positive_labels'] = df[' positive_labels'].apply(cleaning)

  if output_path:
    df.to_csv(output_path)
  return df

In [10]:
df = clean_data_labels('/content/eval_segments_reg_clean.csv', '/content/ontology.json')

In [25]:
timestamps = {row['# YTID']: (row[' start_seconds'], row[' end_seconds']) for i, row in df.iterrows()}

In [3]:
def cleaning(label):
  label = label.replace('"', '').replace(' ', '')
  return ';'.join([clean[i] for i in label.split(';')])

In [14]:
!pip install yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 46.8 MB/s eta 0:00:00


In [15]:
import pandas as pd
from yt_dlp import YoutubeDL
from bs4 import BeautifulSoup

In [4]:
YOUTUBE_URL_PREFIX = "https://www.youtube.com/watch?v="

def download_video(video_url,i):
    opts = {'paths': {'home': f'downloads{i}'}}
    with YoutubeDL(opts) as yt:
        yt.download(video_url)

In [5]:
def download_column(yt_ids, i, stop=None):
    # If stop is None, set it to the length of yt_ids
    stop = len(yt_ids) if stop is None else stop

    # Iterate through the specified number of items
    for yt_id in yt_ids[:stop]:
        download_video(YOUTUBE_URL_PREFIX + yt_id, i)

In [30]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
#ffmpeg_extract_subclip("/content/downloads/Come Mettere La Matita Nera [-0nqfRcnAYE].mp4", 0, 10)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command returned an error


OSError: ffmpeg version 4.2.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libav

In [ ]:
'''from Katna.video import Video
from Katna.writer import KeyFrameDiskWriter
import os

# For windows, the below if condition is must.
if __name__ == "__main__":

  # initialize video module
  vd = Video()

  # number of images to be returned
  no_of_frames_to_returned = 10

  # initialize diskwriter to save data at desired location
  diskwriter = KeyFrameDiskWriter(location="selectedframes")

  # Video file path
  video_file_path = "/content/downloads/VELHO DA TOSSE.wmv [--U7joUcTCo].mp4"

  print(f"Input video file path = {video_file_path}")

  # extract keyframes and process data with diskwriter
  vd.extract_video_keyframes(
       no_of_frames=no_of_frames_to_returned, file_path=video_file_path,
       writer=diskwriter
  )
'''

Input video file path = /content/downloads/VELHO DA TOSSE.wmv [--U7joUcTCo].mp4
Completed processing for :  /content/downloads/VELHO DA TOSSE.wmv [--U7joUcTCo].mp4


In [6]:
import subprocess

def get_audio(id):
  for filename in os.listdir(f'downloads{id}'):
    root, extension = os.path.splitext(filename)
    command = [
      'ffmpeg',
      '-i', f'downloads{id}/{filename}',  # Input file
      '-q:a', '0',  # Video filter: scale to 720p
      '-map', 'a',
      f'audio/{root}.wav'  # Output file
    ]
    print(command)
    subprocess.run(command)

In [23]:
import re

In [21]:
def extract_youtube_ids(filename):
    """
    Extracts YouTube video IDs from a list of filenames.

    Args:
        filenames (list of str): List of filenames containing YouTube IDs.

    Returns:
        list of str: Extracted YouTube IDs.
    """
    pattern = r'\[([-_\w]{11})\]'  # Matches square brackets with 11-character YouTube IDs

    match = re.search(pattern, filename)
    if match:
      return (match.group(1))
    return ''

In [19]:
import os

In [17]:
def get_clips(yt_ids, id):
  filenames = [file for file in os.listdir(f'downloads{id}')]
  title_to_id = {filename : extract_youtube_ids(filename) for filename in filenames}
  for filename in filenames:
    root, extension = os.path.splitext(filename)
    video_id = title_to_id[filename]
    ffmpeg_extract_subclip(f'downloads{id}/{filename}', timestamps[video_id][0], timestamps[video_id][1])
    os.remove(f"downloads{id}/{filename}")

In [29]:
import os
import pandas as pd
import numpy as np

# Example DataFrame


# Number of chunks
n_chunks = 3

# Split into chunks
chunks = np.array_split(df.head(6), n_chunks)


def process_chunck(df, id):
  yt_ids = df['# YTID']
  download_column(yt_ids, id)
  get_clips(yt_ids, id)
  get_audio(id)


# Each chunk is a DataFrame
for i, chunk in enumerate(chunks):
  process_chunck(chunk, i)


[youtube] Extracting URL: https://www.youtube.com/watch?v=--4gqARaEJE
[youtube] --4gqARaEJE: Downloading webpage


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


[youtube] --4gqARaEJE: Downloading ios player API JSON
[youtube] --4gqARaEJE: Downloading mweb player API JSON
[youtube] --4gqARaEJE: Downloading m3u8 information
[info] --4gqARaEJE: Downloading 1 format(s): 136+251
[download] Destination: downloads0/Miniature, Standard, Teacup Dachshund, Puppies, For, Sale, In, New Jersey, NJ, PA, DE, MD,CT [--4gqARaEJE].f136.mp4
[download] 100% of    7.92MiB in 00:00:00 at 14.45MiB/s  
[download] Destination: downloads0/Miniature, Standard, Teacup Dachshund, Puppies, For, Sale, In, New Jersey, NJ, PA, DE, MD,CT [--4gqARaEJE].f251.webm
[download] 100% of  774.71KiB in 00:00:00 at 3.78MiB/s   
[Merger] Merging formats into "downloads0/Miniature, Standard, Teacup Dachshund, Puppies, For, Sale, In, New Jersey, NJ, PA, DE, MD,CT [--4gqARaEJE].mkv"
Deleting original file downloads0/Miniature, Standard, Teacup Dachshund, Puppies, For, Sale, In, New Jersey, NJ, PA, DE, MD,CT [--4gqARaEJE].f251.webm (pass -k to keep)
Deleting original file downloads0/Miniatur